# **Templates - Making Prompts Dynamic and Reusable**

## **What's Covered?**
1. Introduction to Templates
2. 

## **Introduction to Templates**

### **What are Prompt Templates?**
- Writing **static strings as prompts quickly becomes unmanageable**. We need a way to inject dynamic information. This is where Prompt Template comes in.
- These are the objects that help you **construct prompts dynamically by accepting input variables**. Think of them as blueprints for your prompts.
- Templates **offer a more systematic approach to passing in variables to prompts** for models, instead of using f-string literals or .format() calls. The PromptTemplate converts these into function parameter names that we can pass in.

<img src="images/langchain_model_io.jpg">

### **Why they are crucial?**
1. **Reusability:** Define a template once and use it for many different inputs.
2. **Consistency:** Ensure your prompts follow a specific structure every time, which helps LLMs perform better.
3. **Readability:** Makes your code cleaner by separating prompt logic from other code.
4. **Parameterization:** Easily insert varying information into the prompt.


### **Types of Prompt Templates**
1. **PromptTemplate:** Used for generic text completion models or when your prompt is a single string.
2. **ChatPromptTemplate:** Specifically designed for chat models, allowing you to define sequences of `HumanMessage`, `AIMessage`, and `SystemMessage` templates. This is generally preferred for modern LLMs as most are fine-tuned for chat.

### **Input and Output**
- We can call a template with **.invoke()** method.
- **Input** to the Prompt Template should be a **dictionary** containing raw user inputs.
- **Output** of a Prompt Template will be a **string** or **list of chat messages**.

## **PromptTemplate**

Used for generic text completion models or when your prompt is a single string.

**Prompt Template**  
- Prompt Templates are used to convert raw user input to a better input to the LLM.
- Templates allow us to easily configure and modify our input prompts to LLM calls.
- A template may include instructions, few-shot examples, and specific context and questions appropriate for a given task.
- LangChain provides tooling to create and work with prompt templates.
- LangChain strives to create model agnostic templates to make it easy to reuse existing templates across different language models.
- Typically, language models expect the prompt to either be a string or else a list of chat messages.

### **Creating a PromptTemplate by Direct Instantiation**

You can also instantiate PromptTemplate **directly by passing the template string** and **explicitly listing the input_variables**.

In [14]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate(template="Tell me a {adjective} joke about {content}.")

prompt_template

PromptTemplate(input_variables=['adjective', 'content'], input_types={}, partial_variables={}, template='Tell me a {adjective} joke about {content}.')

In [15]:
# Let's check the input variables

prompt_template.input_variables

['adjective', 'content']

### **Creating a PromptTemplate using .from_template()**

`PromptTemplate.from_template(string_with_placeholders)` helps us create a PromptTemplate object from a string with placeholders (e.g., "Tell me about {topic} in 200 words.").

In [2]:
from langchain_core.prompts import PromptTemplate

# Creating a prompt template with input variables
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)

prompt_template

PromptTemplate(input_variables=['adjective', 'content'], input_types={}, partial_variables={}, template='Tell me a {adjective} joke about {content}.')

In [3]:
# Let's check the input variables

prompt_template.input_variables

['adjective', 'content']

### **Passing values to placeholder using .format()**

**.format()** converts the PromptTemplate to a `String`

In [6]:
# format() returns a string

prompt = prompt_template.format(adjective="funny", content="chickens")

print(type(prompt))
print(prompt)

<class 'str'>
Tell me a funny joke about chickens.


### **Passing values to placeholder using .format_prompt()**

**.format_prompt()** converts the PromptTempate to `StringPromptValue`. 

`PromptValues` can be converted to a string and list of messages with the help of `to_string()` and `to_messages()` respectively.

<img src="images/langchain_LCEL.JPG">


In [7]:
# format_prompt() returns a string i.e. StingPromptValue
# PromptValue can be converted to either Strings or Messages

prompt = prompt_template.format_prompt(adjective="funny", content="chickens")

print(type(prompt))
print(prompt)

<class 'langchain_core.prompt_values.StringPromptValue'>
text='Tell me a funny joke about chickens.'


In [11]:
## We can use StingPromptValue and convert it to string using .to_string()

prompt_string = prompt.to_string()

print(type(prompt_string))
print(prompt_string)

<class 'str'>
Tell me a funny joke about chickens.


In [10]:
## We can use StingPromptValue and convert it to List of ChatMessages using .to_messages()

prompt_messages = prompt.to_messages()

print(type(prompt_messages))
print(prompt_messages)

<class 'list'>
[HumanMessage(content='Tell me a funny joke about chickens.', additional_kwargs={}, response_metadata={})]


### **Passing values to placeholder using .format_messages()**

**.format_message()** converts the ChatPromptTemplate to list of `ChatMessages`. **It won't work with the PromptTemplate**.

In [20]:
try:
    prompt = prompt_template.format_messages(adjective="funny", content="chickens")
except:
    print("AttributeError: 'PromptTemplate' object has no attribute 'format_messages'.")

AttributeError: 'PromptTemplate' object has no attribute 'format_messages'.


### **partial_variables**

**partial_variables** in prompt templates are a powerful feature that allows you to "pre-fill" some of the placeholders in your prompt template.

Think of `partial_variables` as **default function parameters**.

**This is particularly useful in scenarios where:**
1. **Some variables are static or known well in advance:** For example, a system instruction that's always the same, or a fixed format_instructions string from an OutputParser. This is covered later in the next chapter (i.e. 4. Output Parsing).
2. **You're building complex chains where some information becomes available earlier:** You can partial a prompt as information becomes available, reducing the number of variables you need to pass around later in the chain.
3. **You need to inject dynamic information from a function:** Instead of a static string, you can provide a function that will be called at the time of formatting to get the value for that variable. This is great for things like current date/time.

In [16]:
from langchain_core.prompts import PromptTemplate

# Creating a prompt template with input variables
prompt_template = PromptTemplate(
    template="Tell me a {adjective} joke about {content}.",
    partial_variables={"adjective": "funny"}
)

prompt_template

PromptTemplate(input_variables=['content'], input_types={}, partial_variables={'adjective': 'funny'}, template='Tell me a {adjective} joke about {content}.')

In [17]:
prompt_template.format(content="chickens")

'Tell me a funny joke about chickens.'

In [18]:
prompt_template.format(content="chickens", adjective="dark")

'Tell me a dark joke about chickens.'

## **ChatPromptTemplate**

The prompt to chat models is a list of chat messages.

Each chat message is associated with content, and an additional parameter called role. For example, in the OpenAI Chat Completions API, a chat message can be associated with an AI assistant, a human or a system role.

### **ChatPromptTemplate and from_template()**

In [10]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_template(
    "What is {topic}?"
)

In [11]:
chat_template.input_variables

['topic']

### **ChatPromptTemplate - .format(), .format_messages() and .format_prompt()**

Templates offer a more systematic approach to passing in variables to prompts for models, instead of using f-string literals or .format() calls. The PromptTemplate converts these into function parameter names that we can pass in.

- .format(): Converts the PromptTemplate to `String`
- .format_message(): Converts the PromptTemplate to list of `ChatMessages`
- .format_prompt(): Converts the PromptTempate to `ChatPromptValue`. `PromptValues` can be converted to both LLM (to string) inputs and ChatModel (to messages) inputs. On this we can apply `to_messages()` or `to_string()`.

In [12]:
# format() returns a string

prompt = chat_template.format(topic="machine learning")

print(type(prompt))
print(prompt)

<class 'str'>
Human: What is machine learning?


In [13]:
# fomat_messages() return list of chat messages

prompt = chat_template.format_messages(topic="machine learning")

print(type(prompt))
print(prompt)

<class 'list'>
[HumanMessage(content='What is machine learning?', additional_kwargs={}, response_metadata={})]


In [14]:
# format_prompt() returns a chat here i.e. ChatPromptValue()
# PromptValue can be converted to either Strings or Chat Messages

prompt = chat_template.format_prompt(topic="machine learning")

print(type(prompt))
print(prompt)

<class 'langchain_core.prompt_values.ChatPromptValue'>
messages=[HumanMessage(content='What is machine learning?', additional_kwargs={}, response_metadata={})]


## **ChatPromptTemplate and from_messages()**

In [15]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

In [16]:
chat_template.input_variables

['name', 'user_input']

In [17]:
chat_template.format(name="Bob", user_input="What is your name?")

"System: You are a helpful AI bot. Your name is Bob.\nHuman: Hello, how are you doing?\nAI: I'm doing well, thanks!\nHuman: What is your name?"

In [18]:
# format_messages() returns chat messages 

chat_template.format_messages(name="Bob", user_input="What is your name?")

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})]

In [19]:
# format_prompt() returns a chat here i.e. ChatPromptValue()

chat_template.format_prompt(name="Bob", user_input="What is your name?")

ChatPromptValue(messages=[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})])

## **ChatPromptTemplate, partial_variables and .invoke()**

In [20]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate(
    messages = [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ], 
    partial_variables={"name": "Alice"}
)

chat_template

ChatPromptTemplate(input_variables=['user_input'], input_types={}, partial_variables={'name': 'Alice'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], input_types={}, partial_variables={}, template='You are a helpful AI bot. Your name is {name}.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Hello, how are you doing?'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="I'm doing well, thanks!"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], input_types={}, partial_variables={}, template='{user_input}'), additional_kwargs={})])

In [21]:
raw_input = {"user_input": "What is your name?"}

chat_template.invoke(raw_input)

ChatPromptValue(messages=[SystemMessage(content='You are a helpful AI bot. Your name is Alice.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})])

In [22]:
print(chat_template.invoke(raw_input).to_string())

System: You are a helpful AI bot. Your name is Alice.
Human: Hello, how are you doing?
AI: I'm doing well, thanks!
Human: What is your name?


In [23]:
print(chat_template.invoke(raw_input).to_messages())

[SystemMessage(content='You are a helpful AI bot. Your name is Alice.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})]


## **Passing values to placeholders using .invoke()**

`PromptTemplate` and `ChatPromptTemplate` implement the Runnable interface, the basic building block of the **LangChain Expression Language (LCEL)**. This means they support `invoke`, `ainvoke`, `stream`, `astream`, `batch`, `abatch`, `astream_log` calls.

**Using `invoke()`:**  
`PromptTemplate` **accepts a dictionary (of the prompt variables)** and returns a `StringPromptValue`.  

A `ChatPromptTemplate` **accepts a dictionary** and returns a `ChatPromptValue`.

## **.pretty_print() Messages**

In [25]:
for msg in chat_template.invoke(raw_input).to_messages():
    msg.pretty_print()

================================ System Message ================================

You are a helpful AI bot. Your name is Alice.
================================ Human Message =================================

Hello, how are you doing?
================================== Ai Message ==================================

I'm doing well, thanks!
================================ Human Message =================================

What is your name?


## **Use Case**

**Designing ChatPromptTemplate using SystemMessagePromptTemplate, HumanMessagePromptTemplate and AIMessagePromptTemplate**

In [26]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate

# System Template
system_prompt_template = SystemMessagePromptTemplate.from_template("You are a helpful AI bot. Your name is {name}.")

# Human Template
human_prompt_template = HumanMessagePromptTemplate.from_template("Hello, how are you doing?")

# AI Template
ai_prompt_template = AIMessagePromptTemplate.from_template("I'm doing well, thanks!")

# Human Template
human_prompt_template_input = HumanMessagePromptTemplate.from_template("{user_input}")

# Compile a chat prompt
chat_template = ChatPromptTemplate(
    messages=[ system_prompt_template, 
               human_prompt_template, 
               ai_prompt_template, 
               human_prompt_template_input ],
    partial_variables={"name": "Alice"}
)

In [28]:
raw_input = {"user_input": "What is your name?"}

for msg in chat_template.invoke(raw_input).to_messages():
    msg.pretty_print()

================================ System Message ================================

You are a helpful AI bot. Your name is Alice.
================================ Human Message =================================

Hello, how are you doing?
================================== Ai Message ==================================

I'm doing well, thanks!
================================ Human Message =================================

What is your name?


### **In above implementation, we can use HumanMessage and AIMessage as shown below:**

In [29]:
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# System Template
system_prompt_template = SystemMessagePromptTemplate.from_template("You are a helpful AI bot. Your name is {name}.")

# Human Template
human_prompt = HumanMessage("Hello, how are you doing?")

# AI Template
ai_prompt = AIMessage("I'm doing well, thanks!")

# Human Template
human_prompt_template = HumanMessagePromptTemplate.from_template("{user_input}")

# Compile a chat prompt
chat_template = ChatPromptTemplate(
    messages=[system_prompt_template, human_prompt, ai_prompt, human_prompt_template]
)

print(chat_template.input_variables)

['name', 'user_input']


In [30]:
raw_input = {"name": "Bob", "user_input": "What is your name?"}

for msg in chat_template.invoke(raw_input).to_messages():
    msg.pretty_print()

================================ System Message ================================

You are a helpful AI bot. Your name is Bob.
================================ Human Message =================================

Hello, how are you doing?
================================== Ai Message ==================================

I'm doing well, thanks!
================================ Human Message =================================

What is your name?


## **MessagesPlaceholder**

A placeholder which can be used to pass in a list of messages.

In [31]:
from langchain_core.prompts import MessagesPlaceholder

prompt = MessagesPlaceholder("chat_history")
prompt.format_messages() # raises KeyError

KeyError: 'chat_history'

In [32]:
prompt = MessagesPlaceholder("chat_history", optional=True)

prompt.format_messages() # returns empty list []

[]

In [33]:
message_list = prompt.format_messages(
                chat_history=[
                    ("system", "You are an AI assistant."),
                    ("human", "Hello!"),
                ]
)

message_list

[SystemMessage(content='You are an AI assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello!', additional_kwargs={}, response_metadata={})]

In [34]:
for msg in message_list:
    msg.pretty_print()

================================ System Message ================================

You are an AI assistant.
================================ Human Message =================================

Hello!


In [35]:
# Limiting the number of messages

prompt = MessagesPlaceholder("chat_history", optional=True, n_messages=1)

message_list = prompt.format_messages(
                chat_history=[
                    ("system", "You are an AI assistant."),
                    ("human", "Hello!"),
                ]
)

for msg in message_list:
    msg.pretty_print()

================================ Human Message =================================

Hello!


In [36]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Compile a chat prompt
chat_template = ChatPromptTemplate(
    messages=[ ("system", "You are a helpful AI bot. Your name is {name}."), 
     MessagesPlaceholder(variable_name="chat_history", optional=True), 
     ("human", "{user_input}") ]
)

chat_template

ChatPromptTemplate(input_variables=['name', 'user_input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemM

In [37]:
raw_input = {"name": "Bob", "user_input": "What is your name?"}

chat_template.invoke(raw_input)

ChatPromptValue(messages=[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})])

In [38]:
# Human Message
human_prompt = HumanMessage(content="Hello, how are you doing?")

# AI Message
ai_prompt = AIMessage(content="I'm doing well, thanks!")

In [39]:
raw_input = {"name": "Bob", "user_input": "What is your name?", "chat_history": [human_prompt, ai_prompt]}

chat_template.invoke(raw_input)

ChatPromptValue(messages=[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})])

In [40]:
chat_template.invoke(raw_input).to_messages()

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})]